In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import itertools
from collections import OrderedDict
from functools import partial

from sklearn.datasets import load_svmlight_file


from links import LinksClassifier
from logit import LogisticRegressionPairwise, LogisticRegression


from sklearn.model_selection import ParameterGrid, StratifiedShuffleSplit, GridSearchCV

from tqdm import tqdm_notebook as tqdm

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
datafiles_toy = [
    r'data/diabetes_scale.libsvm',
    r'data/australian_scale.libsvm',
    r'data/breast-cancer_scale.libsvm',
    r'data/german.numer_scale.libsvm',
    r'data/ionosphere_scale.libsvm',
    r'data/liver-disorders_scale.libsvm',
    r'data/heart_scale.libsvm',
]

In [5]:
def loader(name):
    from sklearn.datasets import load_svmlight_file
    from scipy.sparse import issparse
    filename = 'data/%s.libsvm' % name
    if not name in globals():
        X, y = load_svmlight_file(filename)
        if issparse(X):
            X = X.toarray()
        globals()[name] = (X, y)
    return globals()[name]

In [6]:
datasets = OrderedDict([(os.path.split(f)[-1].replace('.libsvm', ''),
                             partial(loader, os.path.split(f)[-1].replace('.libsvm', '')))
                            for f in datafiles_toy])

In [7]:
X, y= datasets.values()[0]()

In [ ]:
def split_dataset(X,y, percent_labels, percent_links):
    X1, X2, z = sample_links_random(X, y, percent_links)
    n_labels = int(len(y) * percent_labels)
    labels_choice = next(StratifiedShuffleSplit(n_splits=1, train_size=percent_labels).split(X, y))[0]
    return X[labels_choice], y[labels_choice], X1, X2, z

In [8]:
X_r,y_r, X1, X2, z = split_dataset(X, y, percent_labels=0.4, percent_links=0.3)

NameError: name 'split_dataset' is not defined